In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import pandas as pd
from finbert_embedding.embedding import FinbertEmbedding
from transformers import BertTokenizer, BertForSequenceClassification
import numpy as np
from tqdm import tqdm
import glob
import os.path
from collections import Counter

In [2]:
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"

In [3]:
finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

In [4]:
def extract_texts(start, N):

    texts = []
    first_sentence = []
    articles = []
    ID = []
    # Import html tickers from data/ticker
    dir_path = "data/text/"
    list_tickers = os.listdir(dir_path)
    list_tickers.remove('.DS_Store') # for mac
    #list_tickers.remove('id_to_date')
    print(list_tickers)
    
    for ticker in list_tickers:
        list_articles = glob.glob("data/text/"+ticker+"/*")[start:start + N]
        display(list_articles)
        for s in list_articles:
            with open(s) as f:
            
                x = re.sub("data/text/"+str(ticker)+"/","",s)
                #x = re.sub(".html",'',x)
                display(x)
                articles.append(x)
                t = f.read()
                texts.append(t)
        ID = [s.replace("data/text/"+str(ticker)+"/", "") for s in list_articles]
        #display(texts)
    return texts, ID


In [5]:
def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]

    #sentences = [x for x in sentences if len(x.split(' '))>5]
    return sentences


In [6]:
def finbert_texts(n_texts, texts):
    labels = {0:'neutral', 1:'positive',2:'negative'}
    data = np.zeros((n_texts,3))

    for j in tqdm(range(n_texts)):
        print(j)
        sentences = split_into_sentences(texts[j])
        sentences = texts[j].split('\n \n')
        sentences = [x for x in sentences if len(x.split(' '))>4]
        #print(len(sentences))
        inputs = tokenizer(sentences, return_tensors="pt", padding=True,truncation=True,max_length=512)


        outputs = finbert(**inputs)[0]
        i = np.argmax(outputs.detach().numpy(), axis = 1)
        sentiment_counter = list(Counter(i).values())
        data[j,:] = sentiment_counter
        
        
    return data

In [7]:
# Nombre d'articles
N = 100
start = 0
texts, ID = extract_texts(start, N)



  #  df_sentiment.at[index,"ID"] = list_articles[index]
   # df_sentiment.at[index,"ID"] = df_sentiment["ID"].str.replace("data/earning_call/", "")
#ID = 

['CTAS', 'AMZN', 'CNP', 'CAT', 'AAPL', 'AAP', 'CMCSA', 'ANET', 'BRO', 'CAH', 'APTV', 'AAL', 'CDW', 'ABC', 'ALLE', 'GOOG', 'MMM', 'BF.B', 'AMT', 'ADI', 'CAG', 'CNC', 'CME', 'BIO', 'CVX', 'MO', 'CPT', 'CDAY', 'BK', 'BLK', 'CDNS', 'CZR', 'CTSH', 'APD', 'AMAT', 'AKAM', 'AWK', 'CI', 'ANSS', 'CRL', 'LNT', 'BAC', 'CF', 'BBWI', 'BBY', 'BAX', 'GOOGL', 'ALB', 'CCL', 'AIZ', 'ALL', 'BSX', 'ALK', 'CTLT', 'ABBV', 'AFL', 'ACN', 'ATVI', 'CINF', 'AEP', 'CHRW', 'COF', 'ALGN', 'T', 'CBRE', 'CHTR', 'A', 'AIG', 'CFG', 'AEE', 'CMG', 'ADP', 'AMD', 'BWA', 'BXP', 'KMX', 'CHD', 'AME', 'CMA', 'CMS', 'ABT', 'ADM', 'AMP', 'CARR', 'AZO', 'BALL', 'BA', 'APA', 'CPB', 'BIIB', 'APH', 'AMGN', 'CERN', 'AVB', 'ANTM', 'CSCO', 'AVY', 'BR', 'KO', 'AXP', 'AVGO', 'ARE', 'CTXS', 'CBOE', 'ADSK', 'AMCR', 'BMY', 'CL', 'BDX', 'BKNG', 'ATO', 'CB', 'CE', 'BKR', 'TECH', 'ADBE', 'C', 'AON', 'ABMD', 'CLX', 'COP', 'WRB', 'AOS', 'AES', 'SCHW', 'STZ']


['data/text/CTAS/4333164',
 'data/text/CTAS/259823',
 'data/text/CTAS/126709',
 'data/text/CTAS/4188578',
 'data/text/CTAS/2767535',
 'data/text/CTAS/4457710',
 'data/text/CTAS/4476458',
 'data/text/CTAS/447361',
 'data/text/CTAS/4395870',
 'data/text/CTAS/226402',
 'data/text/CTAS/4275325',
 'data/text/CTAS/4313028',
 'data/text/CTAS/3532646',
 'data/text/CTAS/49974',
 'data/text/CTAS/1908781',
 'data/text/CTAS/4109695',
 'data/text/CTAS/69302',
 'data/text/CTAS/4250415',
 'data/text/CTAS/194490',
 'data/text/CTAS/315181',
 'data/text/CTAS/4008834',
 'data/text/CTAS/4208277',
 'data/text/CTAS/280333',
 'data/text/CTAS/2098993',
 'data/text/CTAS/3958734',
 'data/text/CTAS/1076581',
 'data/text/CTAS/3989634',
 'data/text/CTAS/2529425',
 'data/text/CTAS/3332635',
 'data/text/CTAS/295453',
 'data/text/CTAS/724791',
 'data/text/CTAS/4293380',
 'data/text/CTAS/3012016',
 'data/text/CTAS/1704592',
 'data/text/CTAS/4133386',
 'data/text/CTAS/163087',
 'data/text/CTAS/215489',
 'data/text/CTAS

'4333164'

'259823'

'126709'

...

'4034667'

'4209963'

'4397840'

'262476'

'4160117'

In [8]:
 if __name__ == "__main__":
    #print(repr(texts[0]))
    data = finbert_texts(N, texts)
    print(data)
   

  0%|                                                   | 0/100 [00:00<?, ?it/s]

0


  1%|▍                                          | 1/100 [00:20<33:39, 20.40s/it]

1


  2%|▊                                        | 2/100 [01:21<1:12:16, 44.25s/it]

...

55


 56%|█████████████████████▎                | 56/100 [1:13:29<1:01:32, 83.92s/it]

56


 56%|██████████████████████▍                 | 56/100 [1:15:33<59:22, 80.96s/it]


ValueError: could not broadcast input array from shape (2,) into shape (3,)

In [ ]:
df_data = pd.DataFrame(data, columns= ["Neutral","Positive", "Negative"])


df_data["Total"] = df_data["Neutral"] + df_data["Positive"] + df_data["Negative"]
df_data["Difference"] = df_data["Positive"] - df_data["Negative"]
df_data["P_N ratio"] = df_data["Positive"] / df_data["Negative"]
df_data["Difference ratio"] = df_data["Difference"] / df_data["Total"]

df_data["Decile"] = 1 + df_data["Difference ratio"].transform(lambda y: pd.qcut(y, 10, labels=False))

df_data = np.round(df_data, 5)

#total = nb_neutral + nb_positive + nb_negative
#difference = nb_positive - nb_negative
#P_N = nb_positive/nb_negative
#difference_ratio = difference / total
#df_data["ID"] = texts
#df_data["ID"] = 

df_data["ID"] = ID
first_column = df_data.pop("ID")
df_data.insert(0, "ID", first_column)

long = list(df_data.loc[df_data['Decile'].isin([10,9])]["ID"])
short = list(df_data.loc[df_data['Decile'].isin([1,2])]["ID"])

display(df_data)


#display(short)
#short

In [ ]:
display(short)
display(long)